# 第四章 用搜索树的方法玩游戏
本章内容
- 用最小最大算法寻找最佳走法
- 对最小最大树搜索减支进行加速
- 将蒙特卡罗树搜索用于游戏

## 4.1 游戏分类
- 确定的和非确定的
- 完全信息和隐藏信息

||确定的|非确定的|
|---|---|---|
|完全信息|围棋，象棋|Backgammon|
|隐藏信息|Battleship，Stratego|扑克，Scrabble|

## 4.2 用最小最大搜索预测对手走法
## 4.3 解决tic-tac-toe：最小最大的例子
首先，定义表示游戏三种结果的枚举：win, loss, 或者 draw。

In [2]:
import enum
class GameResult(enum.Enum):
    loss = 1
    draw = 2
    win = 3

假定已经有函数`best_result`，给定棋局就可以告诉你玩家可以在该棋局下获得的最好结果。
如果玩家可以保证赢棋——不管次序如何，也不管有多复杂，`best_result`函数会返回`GameResult.win`。
如果玩家可以保证和棋，函数返回`GameResult.draw`。否则函数就会返回`GameResult.loss`。
如果假定该函数已经存在，选择招数的函数就很容易写了：遍历所有可能招数，调用`best_result`，选择自己能够获得最佳结果的步数。多个招数可能会导致相同的结果，这种情况下可以随机从中选择。
下列程序清单给出了如何实现该函数。

In [3]:
class MinimaxAgent(Agent):
    def select_move(self, gamestate):
        winning_moves = []
        draw_moves = []
        losing_moves = []
        for possible_move in game_state.legal_moves():
            next_state = game_state.apply_move(possible_move)
            opponent_best_outcome = best_result(next_state)
            our_best_outcome = reverse_game_reslt(opponent_best_result)
            if our_best_outcome == GameResult.win:
                winning_moves.append(possible_move)
            elif our_best_outcome == GameResult.draw:
                draw_moves.append(possible_move)
            else:
                losing_moves.append(possible_move)
        if winning_moves:
            return random.choice(winning_moves)
        if draw_moves:
            return random.choice(draw_moves)
        return random.choice(losing_moves)

NameError: name 'Agent' is not defined

下面的问题是如何实现`best-result`。
和前一节说的一样，可以从游戏结束开始然后向后推导。
下面的程序给出了最简单的情况：如果游戏已经结束，则只可能有一个结果，只需返回

In [4]:
def best_result(game_state):
    if game_state.is_over():
        if game_state.winner() == game_state.next_player:
            return GameResult.win
        elif game_state.winner() is None:
            return GameResult.draw
        else:
            return GameResult.loss

如果在游戏中间，就要继续向前搜索。从遍历所有可能招数开始，计算下一个游戏状态。
然后你必须假定对手会尽力对抗你假定的招数。
这样，你可以在新的局面下调用`best_result`。
调用后会告诉你对手在这个新位置可以得到的结果；然后倒过来就可以得到自己的结果。
在考虑的所有招数中，选择可以为你带来结果的最好招数。

In [5]:
best_result_so_far = GameResult.loss
for candidate_move in game_state.legal_moves():
    next_state = game_state.apply_move(candidate_move)
    opponent_best_result = best_result(next_state)
    our_result = reverse_game_result(opponent_best_result)
    if our_result.value > best_result_so_far.value:
        best_result_so_far = our_result
return best_result_so_far

NameError: name 'game_state' is not defined

理论上，该算法可以用于象棋、围棋等一切确定性、完全信息的游戏。但实际上该算法太慢了，对所有这些游戏都不适用。

## 4.4 用剪枝减少搜索空间。
博弈树有两个参数，宽度和深度。
宽度是给定棋局下可以选择的走法，深度是从开始到游戏最终状态的回合数。
### 4.4.1 用棋局评估降低搜索深度
模拟谁领先、领先多少这种感觉的函数就是*棋局评估函数*

In [6]:
def capture_diff(game_state):
    black_stones = 0
    white_stones = 0
    for r in range(1, game_state.board.num_rows +1):
        for c in range(1, game_state.board.num_cols +1):
            p = gotypes.Point(r,c)
            color = game_state.board.get(p)
            if color == gotypes.Player.black:
                black_stones +=1
            elif color == gotypes.Player.white:
                white_stones +=1
    diff = black_stones - white_stones
    if game_state.next_player == gotypes.Player.black:
        return diff
    return -1 * diff